# ANALIZA PODATKOV

In [23]:
import pandas as pd
import analiza_funkcije
import html

filmi = pd.read_csv("filmi.csv")

In [24]:
filmi["naslov"] = filmi["naslov"].map(html.unescape)

In [25]:
filmi["razlika_v_ocenah"] = abs(filmi["ocena_rotten_tomatoes"] / 10 - filmi["ocena_imdb"])

In [26]:
filmi["desetletje"] = (filmi["leto"] // 10) * 10

In [27]:
filmi["visja_ocena"] = filmi.apply(analiza_funkcije.doloci_visjo_oceno, axis=1)

In [28]:
#za narediti:
#katero leto ima koliko filmov

In [29]:
filmi

,naslov,leto,ocena_rotten_tomatoes,ocena_imdb,razlika_v_ocenah,desetletje,visja_ocena
0,Modern Problems,1981.0,NaN,5.1,NaN,1980.0,ni_podatkov
1,Sharky's Machine,1981.0,86.0,6.4,2.2,1980.0,rotten_tomatoes
2,Absence of Malice,1981.0,81.0,6.9,1.2,1980.0,rotten_tomatoes
3,On Golden Pond,1981.0,93.0,7.6,1.7,1980.0,rotten_tomatoes
4,Richard Pryor: Live on the Sunset Strip,1982.0,40.0,7.6,3.6,1980.0,rotten_tomatoes
...,...,...,...,...,...,...,...
1166,Bad Boys for Life,2020.0,76.0,6.5,1.1,2020.0,rotten_tomatoes
1167,Birds of Prey (And the Fantabulous Emancipatio...,2020.0,79.0,6.1,1.8,2020.0,rotten_tomatoes
1168,Sonic the Hedgehog,2020.0,64.0,6.5,0.1,2020.0,rotten_tomatoes
1169,The Invisible Man,2020.0,92.0,7.1,2.1,2020.0,rotten_tomatoes
